In [1]:
import datetime
import fetcher
import compute
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2020, 1, 1)
#endDate = datetime.date(2024, 8, 12)
endDate = datetime.date.today()
Stocks_list = []#"NVDA", "AAPL", "DKNG", "UBER"

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS\\US_2024-08-13.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]
#stocks = stocks[~stocks["Industry"].isin(["Biotechnology", "Pharmaceuticals: major", "Medical Specialties"])]
for ticker in stocks["Symbol"].tolist()[1:]:
    Stocks_list.append(ticker)

df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

info = {}
RS_scores = {}

RS_rating_test = {}

focus_list_IPO = pd.DataFrame(columns=['ticker','ADR','ADV'])
for ticker in Stocks_list:
    
    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    
    ## Having this in the beginning is to have all the stocks while calculating percentile
    RS_score = compute.RS_score(df["Close"], df_index["Close"])
    RS_scores[ticker] = RS_score 
    
    # IPO stocks in the last 3 years
    if len(df) < 252*2:
        # Avg Volume*Price
        if len(df) > 30:
            ADV = compute.avg_dollar_vol(df, 30)
        
            """if ADV < 10000000:
                continue"""
    
        # ADR%
        if len(df) > 20:
            ADR = compute.ADR(df, 20)
            """if ADR < 2.25:
                continue"""
            new_row = {"ticker":ticker, "ADR": ADR, "ADV": ADV}
            focus_list_IPO = focus_list_IPO.append(new_row, ignore_index=True)
    else:
        # Avg Volume*Price
        ADV = compute.avg_dollar_vol(df, 30)

        # ADR%
        ADR = compute.ADR(df, 20)
        
        #52 week highs
        _52weekHigh = max(df["Close"][-252:])

        # RS
        #RS = compute.RS_line(df, df_index, 252)
        EMA10 = TA.EMA(df, period= 10)
        EMA21 = TA.EMA(df, period= 21)
        SMA50 =  np.convolve(df["Close"], np.ones(50)/50, mode='valid')
        SMA200 = np.convolve(df["Close"], np.ones(200)/200, mode='valid')
        slope50 = compute.linear_regression(SMA50[-6:])
        slope200 = compute.linear_regression(SMA200[-6:])
        #compute.calc_slope(SMA[-42:], 42)
        #print(angle)
        info[ticker] = {"Close": df["Close"][-1], "ADR" : ADR, "ADV" : ADV, "52weekHigh":_52weekHigh, "EMA10": EMA10[-1], "EMA21": EMA21[-1], "SMA50": SMA50[-1], "SMA200": SMA200[-1], "Slope50": slope50, "Slope200": slope200}
        #RS_rating_test[ticker] = compute.RS_rating_new(RS_scores)


# Sorting based on RS_score 
RS_scores = {ticker: RS_score for ticker, RS_score in sorted(RS_scores.items(), key=lambda item: item[1])}


# RS_rating for the stocks
RS_ratings = compute.RS_rating(RS_scores)

# Storing all info in a dataframe
focus = pd.DataFrame(columns=['ticker',"Close", 'ADR','ADV', '52weekHigh', 'RS_ratings', "EMA10", "EMA21","SMA50", "SMA200", "Slope50", "Slope200"])
for ticker, values in info.items():
    new_row = {"ticker":ticker, "Close" : values["Close"], "ADR": values["ADR"], "ADV": values["ADV"], '52weekHigh' : values["52weekHigh"], "RS_ratings": RS_ratings[ticker], "EMA10":values["EMA10"], "EMA21":values["EMA21"], "SMA50": values["SMA50"], "SMA200": values["SMA200"], "Slope50": values["Slope50"], "Slope200": values["Slope200"]}
    focus = focus.append(new_row, ignore_index=True)
    



fetching data from API for SPY


KeyboardInterrupt: 

### Filtering IPO list

In [35]:
final_focus_IPO = focus_list_IPO.loc[(focus['ADR'] >=2.75) & (focus['ADV'] >=10000000)]


### Filtering NON IPO stocks

In [80]:
final_focus_NON_IPO = focus.loc[ (focus['ADR'] >=2.25) & (focus['ADV'] >=10000000) &(focus["RS_ratings"] >= 80) &(focus["Close"] >= focus['SMA200'])  & (focus['SMA50'] >=focus['SMA200']) & (focus['Close'] >=.8*focus['52weekHigh'])& (focus["EMA10"] >= focus['SMA50']) & (focus["EMA21"] >= focus['SMA50'])& (focus['Slope50'] >=0)] #&  &  & (focus['Slope200'] >=0)(focus['RS_ratings'] >= 40) & (focus['Slope50'] >=0) & (focus['Slope200'] >=0)
len(final_focus_NON_IPO["ticker"].values)

320

### Sorting

In [81]:
final_focus_NON_IPO = pd.DataFrame(final_focus_NON_IPO.sort_values(by=['ADV'], ascending=False).to_numpy(), 
                   index=final_focus_NON_IPO.index, columns=final_focus_NON_IPO.columns)

In [82]:
stocks = stocks.replace({'/': '_'}, regex=True)

### Grouping by Sectors & Industries

In [83]:
sectors = {}
industries = {}
for index, row in final_focus_NON_IPO.iterrows():
    exchange = stocks.loc[stocks['Symbol'] == row["ticker"]]["Exchange"].values[0]
    sector = stocks.loc[stocks['Symbol'] == row["ticker"]]["Sector"].values[0]
    #print(sector)
    industry = stocks.loc[stocks['Symbol'] == row["ticker"]]["Industry"].values[0]
    #print(industry)
    if sector not in sectors.keys():
        sectors[sector] = {}
        
    if industry not in sectors[sector].keys():
        #print(industry)
        sectors[sector][industry] = []
    
    #print(sectors)
            #print(sectors[sector])
    sectors[sector][industry].append(row["ticker"])


In [84]:
sectors

{'Technology services': {'Packaged software': ['PLTR',
   'NOW',
   'FICO',
   'MANH',
   'ZETA',
   'CVLT',
   'SMAR',
   'CWAN',
   'SPSC',
   'VERX',
   'ACIW',
   'YOU',
   'AGYS',
   'ALKT',
   'COMP',
   'PLUS'],
  'Internet software_Services': ['SPOT', 'MMYT', 'EVER'],
  'Information technology services': ['VRNS', 'QTWO', 'PBI'],
  'Data processing services': ['GDS', 'AVPT', 'INOD']},
 'Producer manufacturing': {'Industrial machinery': ['GE',
   'CARR',
   'LII',
   'PNR',
   'MOD',
   'GTES',
   'AAON',
   'ITT',
   'BMI',
   'KAI',
   'OII',
   'NPO',
   'CECO'],
  'Auto parts: OEM': ['CMI', 'ALSN'],
  'Electrical products': ['GNRC', 'BDC'],
  'Building products': ['MHK', 'ALLE', 'SPXC'],
  'Metal fabrication': ['MLI'],
  'Miscellaneous manufacturing': ['AWI'],
  'Trucks_Construction_Farm machinery': ['FSS', 'TRN'],
  'Office equipment_Supplies': ['HNI', 'MLKN']},
 'Retail trade': {'Specialty stores': ['CVNA',
   'KMX',
   'AN',
   'ABG',
   'GPI',
   'RUSHA'],
  'Apparel_Foot

In [85]:
bam = []
for sector,values in sectors.items():
    if sector not in ["Finance"]:
        for industry in values.values():
            for ticker in list(industry):
                bam.append(ticker)


In [88]:
print(bam)

['PLTR', 'NOW', 'FICO', 'MANH', 'ZETA', 'CVLT', 'SMAR', 'CWAN', 'SPSC', 'VERX', 'ACIW', 'YOU', 'AGYS', 'ALKT', 'COMP', 'PLUS', 'SPOT', 'MMYT', 'EVER', 'VRNS', 'QTWO', 'PBI', 'GDS', 'AVPT', 'INOD', 'GE', 'CARR', 'LII', 'PNR', 'MOD', 'GTES', 'AAON', 'ITT', 'BMI', 'KAI', 'OII', 'NPO', 'CECO', 'CMI', 'ALSN', 'GNRC', 'BDC', 'MHK', 'ALLE', 'SPXC', 'MLI', 'AWI', 'FSS', 'TRN', 'HNI', 'MLKN', 'CVNA', 'KMX', 'AN', 'ABG', 'GPI', 'RUSHA', 'BURL', 'SFM', 'JWN', 'MMM', 'TMDX', 'RVTY', 'GKOS', 'ICUI', 'SANM', 'MYGN', 'CDNA', 'LMAT', 'ALNY', 'INSM', 'INCY', 'ITCI', 'TGTX', 'CRNX', 'CORT', 'PETQ', 'MIRM', 'VRNA', 'AVDL', 'SLNO', 'NRIX', 'PRAX', 'CRBP', 'VCEL', 'COLL', 'CPRX', 'HROW', 'KNSA', 'MNKD', 'LGND', 'HRMY', 'ARGX', 'RVMD', 'TEVA', 'HALO', 'KRYS', 'RNA', 'AXSM', 'ADMA', 'GERN', 'PTGX', 'DYN', 'HUMA', 'LBPH', 'KYMR', 'DNLI', 'RYTM', 'BCRX', 'CDMO', 'DHI', 'LEN', 'PHM', 'TOL', 'KBH', 'MTH', 'TMHC', 'TPH', 'MHO', 'FTDR', 'GRBK', 'CCS', 'CVCO', 'PATK', 'HAS', 'REVG', 'LZB', 'IMAX', 'DORM', 'GOLD', '

In [14]:
industries_to_concentrate = ['Aerospace & defense',
 'Specialty stores',
 'Food retail',
 'Medical specialties',
 'Pharmaceuticals: other',
 'Pharmaceuticals: generic',
 'Hospital/Nursing management',
 'Services to the health industry',
 'Medical/Nursing services',
 'Financial publishing/Services',
 'Advertising/Marketing services',
 'Multi-line insurance',
 'Real estate investment trusts',
 'Regional banks',
 'Insurance brokers/Services',
 'Real estate development',
 'Tobacco',
 'Food: meat/fish/dairy',
 'Publishing: newspapers',
 'Wireless telecommunications',
 'Specialty telecommunications',
 'Alternative power generation',
 'Industrial specialties',
 'Oil & gas pipelines',
 'Investment trusts/Mutual funds']

In [20]:
final_final_focus_industries = {}
for sector in sectors.values():
    for industry, tickers in sector.items():
        if industry in industries_to_concentrate:
            if industry not in final_final_focus_industries.keys():
                final_final_focus_industries[industry] = []
            for ticker in tickers:
                final_final_focus_industries[industry].append(ticker)


In [21]:
final_final_focus_industries

{'Aerospace & defense': ['LMT',
  'HWM',
  'LHX',
  'AXON',
  'SPR',
  'FTAI',
  'HEI',
  'CW',
  'ERJ',
  'BWXT',
  'RKLB'],
 'Services to the health industry': ['DGX', 'ASTH'],
 'Multi-line insurance': ['PGR',
  'HIG',
  'CINF',
  'AXS',
  'CNO',
  'TRUP',
  'BWIN',
  'GNW',
  'CNA'],
 'Real estate investment trusts': ['AMT',
  'WELL',
  'PSA',
  'SPG',
  'IRM',
  'EXR',
  'EQR',
  'AVB',
  'ESS',
  'UDR',
  'MAA',
  'CPT',
  'DOC',
  'KIM',
  'BXP',
  'AMH',
  'GLPI',
  'REG',
  'COLD',
  'OHI',
  'CUBE',
  'SLG',
  'HR',
  'FRT',
  'VNO',
  'BRX',
  'EGP',
  'FR',
  'STAG',
  'TRNO',
  'EPRT',
  'IRT',
  'SBRA',
  'RITM',
  'CTRE',
  'KRG',
  'CUZ',
  'NSA',
  'SITC',
  'HIW',
  'DEI',
  'ROIC',
  'LXP',
  'CDP',
  'SKT',
  'IIPR',
  'OUT',
  'BNL',
  'UE',
  'AKR',
  'TWO',
  'JBGS',
  'GTY',
  'FCPT',
  'ESRT',
  'NHI',
  'LTC',
  'DEA'],
 'Regional banks': ['FI',
  'TFC',
  'FITB',
  'FCNCA',
  'MTB',
  'WAL',
  'SNV',
  'EWBC',
  'SSB',
  'COLB',
  'PB',
  'PNFP',
  'BFH',
  'C

In [22]:
final_final_focus = []
for industry in list(final_final_focus_industries.values()):
    for ticker in industry:
        final_final_focus.append(ticker)



In [23]:
final_final_focus

['LMT',
 'HWM',
 'LHX',
 'AXON',
 'SPR',
 'FTAI',
 'HEI',
 'CW',
 'ERJ',
 'BWXT',
 'RKLB',
 'DGX',
 'ASTH',
 'PGR',
 'HIG',
 'CINF',
 'AXS',
 'CNO',
 'TRUP',
 'BWIN',
 'GNW',
 'CNA',
 'AMT',
 'WELL',
 'PSA',
 'SPG',
 'IRM',
 'EXR',
 'EQR',
 'AVB',
 'ESS',
 'UDR',
 'MAA',
 'CPT',
 'DOC',
 'KIM',
 'BXP',
 'AMH',
 'GLPI',
 'REG',
 'COLD',
 'OHI',
 'CUBE',
 'SLG',
 'HR',
 'FRT',
 'VNO',
 'BRX',
 'EGP',
 'FR',
 'STAG',
 'TRNO',
 'EPRT',
 'IRT',
 'SBRA',
 'RITM',
 'CTRE',
 'KRG',
 'CUZ',
 'NSA',
 'SITC',
 'HIW',
 'DEI',
 'ROIC',
 'LXP',
 'CDP',
 'SKT',
 'IIPR',
 'OUT',
 'BNL',
 'UE',
 'AKR',
 'TWO',
 'JBGS',
 'GTY',
 'FCPT',
 'ESRT',
 'NHI',
 'LTC',
 'DEA',
 'FI',
 'TFC',
 'FITB',
 'FCNCA',
 'MTB',
 'WAL',
 'SNV',
 'EWBC',
 'SSB',
 'COLB',
 'PB',
 'PNFP',
 'BFH',
 'CBSH',
 'BPOP',
 'HOMB',
 'TBBK',
 'GBCI',
 'RNST',
 'EBC',
 'ABCB',
 'SFBS',
 'FIBK',
 'FBP',
 'AUB',
 'FHB',
 'UCB',
 'KB',
 'HTLF',
 'CATY',
 'SBCF',
 'IBTX',
 'FCF',
 'NBHC',
 'FFBC',
 'BANR',
 'TOWN',
 'BANF',
 'TRMK',
 'EFSC

In [24]:
len(final_final_focus)

172

### Industries in each sector

In [5]:
a = stocks['Industry'].unique()
#for index, row in stocks.iterrows():
sectors = {}
for industry in a:
    sector = stocks.loc[stocks['Industry'] == industry]["Sector"].values[0]
    if sector not in sectors:
        sectors[sector] = []
    sectors[sector].append(industry)

In [13]:
sectors

{'Utilities': ['Gas distributors',
  'Alternative power generation',
  'Water utilities',
  'Electric utilities'],
 'Industrial services': ['Oil & gas pipelines',
  'Oilfield services/Equipment',
  'Environmental services',
  'Engineering & construction',
  'Contract drilling'],
 'Health technology': ['Biotechnology',
  'Pharmaceuticals: other',
  'Pharmaceuticals: major',
  'Medical specialties',
  'Pharmaceuticals: generic'],
 'Finance': ['Real estate development',
  'Investment banks/Brokers',
  'Investment managers',
  'Real estate investment trusts',
  'Insurance brokers/Services',
  'Regional banks',
  'Property/Casualty insurance',
  'Finance/Rental/Leasing',
  'Financial conglomerates',
  'Major banks',
  'Multi-line insurance',
  'Savings banks',
  'Life/Health insurance',
  'Specialty insurance'],
 'Transportation': ['Marine shipping',
  'Trucking',
  'Air freight/Couriers',
  'Other transportation',
  'Airlines',
  'Railroads'],
 'Technology services': ['Packaged software',


In [ ]:
a = {"Materials":["Process industries", "Non-energy minerals"], "Energy":["Energy minerals", 'Oil & gas pipelines', 'Oilfield services/Equipment'], "Industrials": ['Producer manufacturing', 'Industrial services', 'Aerospace & defense', 'Electronic technology', 'Transportation'], "Consumer Discretionary":['Consumer durables', 'Consumer services', 'Retail trade', 'Distribution services'], "Consumer Staples":['Consumer non-durables'], "Health Care":['Health technology', 'Health services']
     , "Financials": ['Finance'], "Information Technology":['Technology services','Semiconductors'], "Communication Services":['Communications'], "Utilities":['Utilities'],"Real Estate":[] }

In [52]:
sectors["Non-energy minerals"]["Precious metals"]

['GOLD',
 'NEM',
 'AEM',
 'KGC',
 'WPM',
 'FNV',
 'GFI',
 'RGLD',
 'HMY',
 'AU',
 'IAG',
 'AGI',
 'EGO']